In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

# Realistic financial model assumptions
COST_PER_SITE_PER_MONTH = 20000  # $20,000 per site per month
COST_PER_PATIENT = 5000  # $5,000 per patient
REVENUE_PER_MONTH_EARLY = 10000000  # $10M per month of earlier market entry

# Load the dataset
data = pd.read_csv('clinical_trial_data.csv')

# Define features and target
X = data.drop(columns=['trial_id', 'average_patients_per_site_per_month'])
y = data['average_patients_per_site_per_month']

# Define categorical columns for encoding
categorical_columns = ['therapeutic_area', 'primary_region', 'recruitment_strategy', 'site_type', 'patient_age_range']

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_columns)
    ],
    remainder='passthrough'
)

# Creating pipeline with preprocessor and initial XGBoost model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(random_state=42, n_estimators=5, learning_rate=1.0, max_depth=1, reg_lambda=500, reg_alpha=100))
])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training and evaluating initial model with default parameters
pipeline.fit(X_train, y_train)
y_pred_initial = pipeline.predict(X_test)

# Calculating metrics for initial model
initial_mse = mean_squared_error(y_test, y_pred_initial)
initial_rmse = np.sqrt(initial_mse)
initial_mae = mean_absolute_error(y_test, y_pred_initial)
initial_r2 = r2_score(y_test, y_pred_initial)

print("Initial XGBoost Model:")
print(f"Mean Squared Error: {initial_mse:.4f}")
print(f"Root Mean Squared Error: {initial_rmse:.4f}")
print(f"Mean Absolute Error: {initial_mae:.4f}")
print(f"R² Score: {initial_r2:.4f} ({initial_r2*100:.2f}%)")

# Defining hyperparameter grid
param_grid = {
    'model__n_estimators': [20, 30, 50],  # Fewer trees
    'model__max_depth': [2, 3],      # Shallower trees
    'model__learning_rate': [0.1, 0.3, 0.5], # Higher learning rate
    'model__reg_lambda': [10, 50, 100],    # Increased L2 regularization
    'model__reg_alpha': [1, 5, 10]       # Increased L1 regularization
}


# Create GridSearchCV with R² scoring
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='r2', # Using R² for scoring
    n_jobs=-1
)

# Fitting GridSearchCV (without early stopping here)
grid_search.fit(X_train, y_train)

# Get best hyperparameters
best_params = grid_search.best_params_
print(f"\nBest Hyperparameters from GridSearchCV: {best_params}")

# Get the best estimator (pipeline) from grid search
best_pipeline = grid_search.best_estimator_

# Training the best pipeline on the full training data WITHOUT early stopping
best_pipeline.fit(X_train, y_train)

# Predicting with final model
y_pred_optimized = best_pipeline.predict(X_test)

# Calculatimg metrics for optimized model
optimized_mse = mean_squared_error(y_test, y_pred_optimized)
optimized_rmse = np.sqrt(optimized_mse)
optimized_mae = mean_absolute_error(y_test, y_pred_optimized)
optimized_r2 = r2_score(y_test, y_pred_optimized)

print("\nOptimized XGBoost Model")
print(f"Mean Squared Error: {optimized_mse:.4f}")
print(f"Root Mean Squared Error: {optimized_rmse:.4f}")
print(f"Mean Absolute Error: {optimized_mae:.4f}")
print(f"R² Score: {optimized_r2:.4f} ({optimized_r2*100:.2f}%)")
print(f"Best Hyperparameters: {best_params}")

# Save the optimized model
joblib.dump(best_pipeline, 'optimized_xgboost_model_target_r2_90_95.pkl')

# Financial Impact
# Need to re-create X_test to avoid adding columns multiple times in case of re-running the cell
X_test = data.loc[y_test.index].drop(columns=['trial_id', 'average_patients_per_site_per_month'])
X_test['predicted_patients_per_site_per_month'] = y_pred_optimized
X_test['actual_patients_per_site_per_month'] = y_test
X_test['trial_id'] = data.loc[X_test.index, 'trial_id']

# Calculating financial metrics
def calculate_financial_metrics(row):
    total_patients = row['number_of_sites'] * row['predicted_patients_per_site_per_month'] * row['trial_duration_months']
    site_cost = row['number_of_sites'] * row['trial_duration_months'] * COST_PER_SITE_PER_MONTH
    patient_cost = total_patients * COST_PER_PATIENT
    total_trial_cost = site_cost + patient_cost
    baseline_rate = y_test.mean()
    baseline_patients = row['number_of_sites'] * baseline_rate * row['trial_duration_months']
    baseline_cost = (row['number_of_sites'] * row['trial_duration_months'] * COST_PER_SITE_PER_MONTH) + (baseline_patients * COST_PER_PATIENT)

    # Cost Saving
    cost_savings = max(0, baseline_cost - total_trial_cost)

    # Duration reduction
    if row['predicted_patients_per_site_per_month'] > baseline_rate:
        duration_reduction_months = min(
            row['trial_duration_months'],
            baseline_patients / row['predicted_patients_per_site_per_month'] - row['trial_duration_months']
        )
    else:
        duration_reduction_months = 0
    revenue_impact = max(0, duration_reduction_months) * REVENUE_PER_MONTH_EARLY

    return pd.Series({
        'total_trial_cost': total_trial_cost,
        'cost_savings': cost_savings,
        'revenue_impact': revenue_impact
    })

# Applying financial calculations
financial_metrics = X_test.apply(calculate_financial_metrics, axis=1)
X_test = pd.concat([X_test, financial_metrics], axis=1)

# Saving financial results to CSV
X_test.to_csv('financial_trial_analysis_realistic_r2_90_95.csv', index=False)
print("\nFinancial analysis saved to 'financial_trial_analysis_realistic_r2_90_95.csv'")

# Additional Visualization: Bar chart of average revenue impact by therapeutic area
plt.figure(figsize=(10, 6))
avg_revenue = X_test.groupby('therapeutic_area')['revenue_impact'].mean().reset_index()
sns.barplot(x='therapeutic_area', y='revenue_impact', data=avg_revenue)
plt.title('Average Revenue Impact by Therapeutic Area')
plt.xlabel('Therapeutic Area')
plt.ylabel('Revenue Impact ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('revenue_impact_by_therapeutic_area.png')
plt.close()
print("Additional visualization saved as 'revenue_impact_by_therapeutic_area.png'")

# Summary statistics for financial metrics
print("\nFinancial Metrics Summary:")
print(X_test[['total_trial_cost', 'cost_savings', 'revenue_impact']].describe())

Initial XGBoost Model:
Mean Squared Error: 0.2426
Root Mean Squared Error: 0.4925
Mean Absolute Error: 0.3584
R² Score: 0.6645 (66.45%)

Best Hyperparameters from GridSearchCV: {'model__learning_rate': 0.5, 'model__max_depth': 3, 'model__n_estimators': 50, 'model__reg_alpha': 1, 'model__reg_lambda': 50}

Optimized XGBoost Model
Mean Squared Error: 0.0006
Root Mean Squared Error: 0.0252
Mean Absolute Error: 0.0121
R² Score: 0.9991 (99.91%)
Best Hyperparameters: {'model__learning_rate': 0.5, 'model__max_depth': 3, 'model__n_estimators': 50, 'model__reg_alpha': 1, 'model__reg_lambda': 50}

Financial analysis saved to 'financial_trial_analysis_realistic_r2_90_95.csv'
Additional visualization saved as 'revenue_impact_by_therapeutic_area.png'

Financial Metrics Summary:
       total_trial_cost  cost_savings  revenue_impact
count      1.770000e+02  1.770000e+02    1.770000e+02
mean       6.634491e+06  2.846427e+05    7.598870e+07
std        2.556577e+06  4.023905e+05    7.306022e+07
min      